In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train', 'test', 'depths.csv', 'train.csv', 'sample_submission.csv']


In [6]:
import numpy as np
import cv2
from tqdm import tqdm
import os

In [10]:
TRAIN_IMAGE_DIR = '../input/train/images/'
TRAIN_MASK_DIR = '../input/train/masks/'
TEST_IMAGE_DIR = '../input/test/images/'

In [11]:
train_data = os.listdir(TRAIN_IMAGE_DIR)
x = [np.array(cv2.imread(TRAIN_IMAGE_DIR + p, cv2.IMREAD_GRAYSCALE), dtype=np.uint8) for p in tqdm(train_data)]
x = np.array(x)/255

y = [np.array(cv2.imread(TRAIN_MASK_DIR + p, cv2.IMREAD_GRAYSCALE), dtype=np.uint8) for p in tqdm(train_data)]
y = np.array(y)/255
print(x.shape,y.shape)

100%|██████████| 4000/4000 [00:04<00:00, 922.71it/s]


(4000, 101, 101) (4000, 101, 101)


In [12]:
x=np.expand_dims(x,axis=3)
y=np.expand_dims(y,axis=3)
print(x.shape,y.shape)

(4000, 101, 101, 1) (4000, 101, 101, 1)


In [14]:
from keras.layers import MaxPooling2D,Conv2D,Dense,Dropout,Input,Conv2DTranspose,Concatenate
from keras.models import Sequential,Model
from keras.optimizers import Adam
import keras
def conv_block(num_layers,inp,units,kernel_size):
    x = input
    for l in range(num_layers): #repeat 32-24-16 ----4 times
        x = Conv2D(units, kernel_size=kernel_size,padding='SAME',activation='relu')(x)
    return x
input = Input(shape=(101,101,1))
cnn1 = conv_block(5,input,32,3)
cnn2 = conv_block(5,input,24,5)
cnn3 = conv_block(5,input,16,7)
cnn4 = conv_block(5,input,8,9)
cnn5 = conv_block(5,input,4,11)
concat = Concatenate()([cnn1,cnn2,cnn3,cnn4,cnn5])

d1 = Conv2D(16,1,activation='relu')(concat)
out = Conv2D(1,1,activation='sigmoid')(d1) #filter_size = 1 ,so that 1x1 filter will scan over for more learning

model = Model(inputs=[input], outputs=[out])
adam=Adam(lr=0.001)
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary() # start_dim=(101,101,1) == #end_dim=(101,101,1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 101, 101, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 101, 101, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 101, 101, 24) 624         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 101, 101, 16) 800         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_16 

In [ ]:
keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
model.fit(x,y,epochs=50,batch_size=128,validation_split=0.2,verbose=True)